In [1]:
if 'google.colab' in str(get_ipython()):
    import sys
    from google.colab import drive, output
    drive.mount('/content/drive')
    !pip install torchaudio
    !pip install wandb --upgrade
    output.clear()
    print("Running on colab")
    %load_ext autoreload
    %autoreload 1
    %cd '/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project'
    root = "/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project"
else:
    print("Running local")
    root = "/home/mirp_ai/Documents/Daniel_Nieto/PhD/AutoEncoders_Ecoacoustics"
    root_path = "/media/mirp_ai/Seagate Desktop Drive/Jaguas_2018"


Running local


In [2]:
import os
import numpy as np

import datetime
from datetime import timedelta

import torch
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
from torchvision.utils import make_grid
from Jaguas_DataLoader_rainless import SoundscapeData

import pandas as pd

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cuda


In [11]:
class df_generator():
    def __init__(self, root_clusters, root_audios, root_recorders, n_clusters_index = 0):
        self.n_clusters_index = n_clusters_index
        self.cluster_names = os.listdir(f"{root_clusters}")
        self.torch_clusters = torch.load(f"{root_clusters}/{self.cluster_names[n_clusters_index]}")
        self.dataframe_clusters = pd.DataFrame(self.torch_clusters)
        self.dataframe_clusters = self.dataframe_clusters.transpose()
        self.dataframe_audios = pd.read_csv(f"{root_audios}", index_col=0)
        self.dataframe_recorders = pd.read_csv(f"{root_recorders}",sep = ";", index_col = "Recorder")
        self.df_clusters_len = len(self.dataframe_clusters)
        self.df_recorders_len = len(self.dataframe_recorders)
        self.n_clusters = np.arange(len(self.torch_clusters))

    def show_clusters(self, keyword=None):
        if keyword != None:
            for i in range(len(self.cluster_names)):
                if keyword in self.cluster_names[i]:
                    print(f"{i}: {self.cluster_names[i]}")
        else:
            for i in range(len(self.cluster_names)):
                print(f"{i}: {self.cluster_names[i]}")

    def init_clusters(self):
        for cluster in self.n_clusters:
            self.dataframe_recorders[f"Cluster {cluster}"] = 0
    
    def create_clusters(self, count_cluster=False):
        self.init_clusters()
        for cluster in self.n_clusters:
            for i in range(self.df_clusters_len):  
                if self.dataframe_clusters.iloc[i][cluster] != None:
                    self.dataframe_recorders.loc[self.dataframe_clusters.iloc[i][cluster].split("_")[0], f"Cluster {cluster}"] +=1
                else:
                    pass
        if count_cluster == True:
            self.count_cluster_data()
        else:
            pass
        
        return self.dataframe_recorders
    
    def count_cluster_data(self):
        self.dataframe_recorders["Total_Clustering_Audios"] = 0
        for recorder in range(self.df_recorders_len):
            clusters = [f"Cluster {i}" for i in range(0, len(self.n_clusters))]
            clusters.append("Total_Clustering_Audios")
            self.dataframe_recorders.iloc[recorder, -1] = self.dataframe_recorders[clusters].iloc[recorder].sum()
    
    def GLM_dataframe(self):
        GLM = self.dataframe_recorders.copy()
        columns = [f"Cluster {i}" for i in range(0, len(self.n_clusters))]
        columns.append("Total_Clustering_Audios")
        GLM = GLM[columns]
        for cluster in range(len(columns[0:-1])):
            for recorder in range(self.df_recorders_len):
                GLM.iloc[recorder, cluster] = GLM.iloc[recorder, cluster]/GLM.iloc[recorder, -1]
        return GLM
    
    def regions(self):
        self.dataframe_audios["Region"] = 0
        for audio in range(len(self.dataframe_audios)):   
            self.dataframe_audios.loc[audio,"Region"] = self.dataframe_recorders.loc[self.dataframe_audios.loc[audio, "Recorder"]].loc["Region"]
        return self.dataframe_audios
    
    def save(self, root_save=None):
        if root_save != None:
            os.makedirs(f"{self.root_save}/dataframes", exist_ok=True)
            self.dataframe_recorders.to_csv(f"{self.root_save}/dataframes/dataframe_{self.cluster_names[self.n_clusters_index]}_recorders.csv")
            self.dataframe_recorders.to_csv(f"{self.root_save}/dataframes/dataframe_{self.cluster_names[self.n_clusters_index]}_audios.csv") 
            
        else:
            self.dataframe_recorders.to_csv(f"{self.cluster_names[sel.n_clusters_index]}_recorders.csv")
            self.dataframe_audios.to_csv(f"{self.cluster_names[sel.n_clusters_index]}_audios.csv")
    
    def recorders(self):
        return self.dataframe_recorders

In [12]:
root_audios = f"{root}/Jaguas/Complementary_Files/Audios_Jaguas/Audios_Jaguas.csv"
root_recorders = f"{root}/Jaguas/Complementary_Files/df_grabadoras_reg.csv"
root_clusters = f"{root}/Jaguas/temporal/clusters"
ecological_integrity = f"{root}/Jaguas/Complementary_Files/Indice_Integridad_Ecologica.xlsx"
df_EI = pd.read_excel(ecological_integrity)
df_EI.rename(columns={"Sitio":"Recorder"}, inplace=True)
df_EI.set_index("Recorder", inplace=True)

In [13]:
dataframe = df_generator(root_clusters, root_audios, root_recorders, 
                         n_clusters_index=1)
dataframe.create_clusters(count_cluster=True)
GLM = dataframe.GLM_dataframe()
# GLM = GLM.reset_index().set_index(np.arange(len(GLM)))
GLM["Mean"] = df_EI["Mean"]
GLM["Sum"] = df_EI["Sum"]
GLM["Max"] = df_EI["Max"]
GLM["Min"] = df_EI["Min"]
GLM

,Cluster 0,Cluster 1,Cluster 2,Cluster 3,Cluster 4,Total_Clustering_Audios,Mean,Sum,Max,Min
Recorder,,,,,,,,,,
G03,0.151316,0.220614,0.221491,0.263596,0.142982,2280,0.572591,78.444947,0.662041,0.242650
G04,0.114110,0.128834,0.283436,0.396933,0.076687,1630,0.587104,82.194536,0.645013,0.388676
G06,0.042595,0.174612,0.054443,0.343865,0.384485,3545,0.784025,110.547518,0.896008,0.312327
G07,0.032937,0.365873,0.313889,0.150397,0.136905,2520,0.370633,51.147362,0.532298,0.081821
G08,0.197107,0.212397,0.142149,0.247934,0.200413,2420,0.587971,80.552073,0.728520,0.384826
G09,0.031092,0.088515,0.041176,0.445938,0.393277,3570,0.723915,100.624248,0.790780,0.406501
G13,0.204824,0.218182,0.175881,0.170315,0.230798,2695,0.571372,80.563485,0.691484,0.360173
G15,0.027171,0.769748,0.039216,0.091036,0.072829,3570,0.519462,72.724744,0.562418,0.176436
G17,0.245479,0.238630,0.079726,0.164110,0.272055,3650,0.534994,75.434084,0.793996,0.348641


In [21]:
dataframe.show_clusters(keyword="UMAP")

1: kmeans_UMAP_clustering_labels_AE_rainless_5
3: kmeans_UMAP_clustering_labels_PosAE_rainless_3
5: kmeans_UMAP_clustering_labels_PosAE_rainless_10
12: kmeans_UMAP_clustering_labels_PosAE_rainless_5
18: kmeans_UMAP_clustering_labels_AE_rainless_3
20: kmeans_UMAP_clustering_labels_AE_rainless_10
21: kmeans_UMAP_clustering_labels_PosAE_rainless_15
22: kmeans_UMAP_clustering_labels_AE_rainless_15


In [22]:
n_clusters = len(GLM.iloc[0])-5
columns = [f"Cluster {i}" for i in range(0, n_clusters)]
Bar_GLM = GLM[columns]
Bar_GLM["Mean"] = df_EI["Mean"]
# Bar_GLM.index.names = recorders.index
Bar_GLM = Bar_GLM.sort_values(by=["Mean"])
Bar_GLM.drop(columns=["Mean"], inplace=True)
Bar_GLM

/tmp/ipykernel_176922/2880760701.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Bar_GLM["Mean"] = df_EI["Mean"]


,Cluster 0,Cluster 1,Cluster 2,Cluster 3,Cluster 4
Recorder,,,,,
G36,0.001518,0.875142,0.000000,0.025806,0.097533
G34,0.089474,0.677632,0.000987,0.145066,0.086842
G07,0.032937,0.365873,0.313889,0.150397,0.136905
G27,0.061798,0.426592,0.124345,0.192509,0.194757
G41,0.071893,0.119694,0.259656,0.378203,0.170554
G19,0.114980,0.280162,0.090688,0.207827,0.306343
G47,0.233600,0.088000,0.251840,0.303680,0.122880
G49,0.320561,0.135826,0.286293,0.122741,0.134579
G50,0.122976,0.085339,0.322538,0.392123,0.077024


In [59]:
import matplotlib.colors as mcolors
import random
 
def random_color_generator():
    colors = []
    for i in range(n_clusters):
        random_color = random.choice(list(mcolors.CSS4_COLORS.keys()))
        while(random_color in colors):
            random_color = random_color_generator()
        colors.append(random_color)
    return colors
colors = random_color_generator()
    

In [60]:
colors

['wheat', 'orangered', 'gainsboro', 'springgreen', 'lightslategrey']

In [61]:
%matplotlib qt
Bar_GLM.plot(kind='bar', stacked=True, color=colors)

<AxesSubplot: xlabel='Recorder'>

In [ ]:
regiones = dataframe.regions()
regiones

# Using SK-Learn

In [ ]:
from sklearn import linear_model
clf = linear_model.PoissonRegressor()
clf.fit(X,y)

In [ ]:
clf.score(X,y)

# Using StatsModels

In [ ]:
gamma_results.pseudo_rsquared()

In [ ]:
PCA_index = [27, 10, 6, 23]
UMAP_index = [3, 12, 5, 21]
TSNE_index = [29, 15, 17, 13]

In [ ]:
import matplotlib.pyplot as plt
plt.plot(r2_PCA)
plt.plot(r2_UMAP)
plt.plot(r2_TSNE)
plt.legend(["PCA", "UMAP", "TSNE"])
plt.xticks((0,1,2,3),("3", "5", "10", "15"))

In [ ]:
import matplotlib.pyplot as plt
import statsmodels.api as sm
r2_PCA = []
for index in PCA_index:
    dataframe = df_generator(root_clusters, root_audios, root_recorders, 
                         n_clusters_index=index)
    dataframe.create_clusters(count_cluster=True)
    GLM = dataframe.GLM_dataframe()
    GLM = GLM.reset_index().set_index(np.arange(len(GLM)))
    GLM["Mean"] = df_EI["Mean"]
    GLM["Sum"] = df_EI["Sum"]
    GLM["Max"] = df_EI["Max"]
    GLM["Min"] = df_EI["Min"]
    X= GLM[[f"Cluster {i}" for i in range(len(GLM.columns)-6)]]
    X = X.to_numpy()
    y = GLM["Mean"]
    y = y.to_numpy()
    gamma_model = sm.GLM(y, X, family=sm.families.Gamma())
    gamma_results = gamma_model.fit()
    r2_PCA.append(gamma_results.pseudo_rsquared())

In [ ]:
print(r2_TSNE)

In [ ]:
import statsmodels.api as sm
X= GLM[[f"Cluster {i}" for i in range(len(GLM.columns)-6)]]
X = X.to_numpy()
y = GLM["Mean"]
y = y.to_numpy()
gamma_model = sm.GLM(y, X, family=sm.families.Gamma())
gamma_results = gamma_model.fit()
print(gamma_results.summary())

In [ ]:
import statsmodels.api as sm
data = sm.datasets.scotland.load()
data.exog = sm.add_constant(data.exog)
gamma_model = sm.GLM(data.endog, data.exog, family=sm.families.Gamma())
gamma_results = gamma_model.fit()
print(gamma_results.summary())

# SK-Learn example

In [ ]:
from sklearn import linear_model
clf = linear_model.PoissonRegressor()
X = [[1, 2], [2, 3], [3, 4], [4, 3]]
y = [12, 17, 22, 21]
clf.fit(X, y)
clf.score(X, y)
